In [7]:
import os,sys
import math
import numpy as np

import PIL
import torchvision
from torchvision import transforms
import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.nn.functional as F

## reference: [design a shallow neural network for image classification](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

### 1. Network layers

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)
        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.dropout(x) # added dropout
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


### 2. Loss function

In [9]:
criterion = nn.CrossEntropyLoss()

### 3. Optimizer

In [10]:
learning_rate = 0.001
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate, weight_decay = 0.01) # weight decay: L2 penalty

### 4. Train model

In [11]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes an image dataset for PyTorch'
    def __init__(self, list_files, labels, transform = None):
        'Initialization'
        self.labels = labels
        self.list_files = list_files
        self.transform = transform
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_files)
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        file_dir = self.list_files[index]

        # Load data and get label
        image = PIL.Image.open(file_dir)
        if self.transform:
            image = self.transform(image)
        X = image
        y = torch.tensor(self.labels[file_dir])

        return X, y

data_dir = "/mnt/Storage/home/yuzhaowei/.keras/datasets/flower_photos"
labels = {}
daisy_images = [os.path.join((data_dir), "daisy") + "/" + i for i in os.listdir(os.path.join((data_dir), "daisy"))]
rose_images = [os.path.join((data_dir), "roses") + "/" + i for i in os.listdir(os.path.join((data_dir), "roses"))]
sunflowers_images = [os.path.join((data_dir), "sunflowers") + "/" + i for i in os.listdir(os.path.join((data_dir), "sunflowers"))]
samples = np.concatenate([np.random.choice(daisy_images, 500), np.random.choice(rose_images, 500), np.random.choice(sunflowers_images, 500)])

for image in daisy_images:
    labels[image] = 0 # dasiy
for image in rose_images:
    labels[image] = 1 # roses
for image in sunflowers_images:
    labels[image] = 2 # sunflowers

class Rescale():
    """Reize image in a sample to a given size"""
    def __call__(self, image):
        image_resize = np.array(T.Resize(size = (32, 32))(image))
        return(image_resize)
    
# Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8
composed = T.Compose([Rescale(), T.ToTensor()]) # composed two treatments for input images    
        
    
dataset = Dataset(samples, labels, transform = composed)
train_set, test_set = torch.utils.data.random_split(dataset, lengths = [int(1500 * 0.8), int(1500 * 0.2)])

batch_size = 120
n_iters = 100
num_epochs = n_iters / (len(train_set) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6,
         'drop_last' : True # set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size.
         }
train_generator = torch.utils.data.DataLoader(train_set, **params)
test_generator = torch.utils.data.DataLoader(test_set, **params)

import collections
isinstance(train_generator, collections.Iterable)
isinstance(test_generator, collections.Iterable)

10


True

In [12]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_generator):
        # Load images as Variable
        # images = images.requires_grad_()
        images = images.requires_grad_()
        print(images.shape)
        labels = labels
        # print(labels)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = net(images)
        # print(outputs)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_generator:
                # Load images to a Torch Variable
                # images = images.view(-1, 3*224*224).requires_grad_()

                # Forward pass only to get logits/output
                outputs = net(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
Iteration: 10. Loss: 1.1063742637634277. Accuracy: 40.83333206176758
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
Iteration: 20. Loss: 1.105272889137268. Accuracy: 40.0
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
torch.Size([120, 3, 32, 32])
Iterat